In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import pandas as pd
import math

In [2]:
df = pd.read_csv('../2.Feature Engineering/new data/modelling_set.csv')
df.describe()

,MemberID,LabCount_total,LabCount_max,LabCount_min,LabCount_ave,LabCount_months,LabCount_std,DrugCount_total,DrugCount_max,DrugCount_min,...,ProcedureGroup_Count_SO,ProcedureGroup_Count_SMCD,AgeAtFirstClaim,Male,Female,MissSex,MissAge,ClaimsTruncated,TARGET,trainset
count,2.184150e+05,218415.000000,218415.000000,218415.000000,218415.000000,218415.000000,218415.000000,218415.000000,218415.000000,218415.000000,...,218415.000000,218415.000000,218415.000000,218415.000000,218415.000000,218415.000000,218415.0,218415.000000,147473.000000,218415.000000
mean,4.993458e+07,6.471827,3.829439,2.181860,2.918800,1.655033,0.578166,10.387318,2.193531,0.962237,...,0.001699,0.015457,47.750155,0.363524,0.448078,0.188398,0.0,0.054863,0.452951,0.675196
std,2.892479e+07,7.734678,3.440373,2.507408,2.676663,1.906704,1.683426,15.721582,2.271990,1.057978,...,0.047286,0.262061,23.238375,0.481015,0.497298,0.391031,0.0,0.227714,1.573818,0.468303
min,4.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
25%,2.478774e+07,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,35.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
50%,4.993677e+07,4.000000,4.000000,1.000000,2.666667,1.000000,0.000000,3.000000,2.000000,1.000000,...,0.000000,0.000000,45.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.000000
75%,7.515525e+07,10.000000,7.000000,3.000000,5.000000,2.000000,1.527525,14.000000,4.000000,1.000000,...,0.000000,0.000000,65.000000,1.000000,1.000000,0.000000,0.0,0.000000,0.000000,1.000000
max,9.999882e+07,111.000000,10.000000,10.000000,10.000000,12.000000,6.363961,84.000000,7.000000,7.000000,...,4.000000,23.000000,80.000000,1.000000,1.000000,1.000000,0.0,1.000000,15.000000,1.000000


In [3]:
df_train = df[df['trainset']==1]

In [4]:
df_target = df[df['trainset']==0]

In [5]:
m = (df_train.columns != 'TARGET') & (df_train.columns != 'MemberID') & (df_train.columns != 'Year') & (df_train.columns != 'trainset')
X = df_train.loc[:, m]
y = df_train['TARGET']
y.describe()

count    147473.000000
mean          0.452951
std           1.573818
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          15.000000
Name: TARGET, dtype: float64

In [6]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = MinMaxScaler(feature_range=(0,1)).fit(X)
X_scl = scaler.transform(X)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_scl, y, test_size=0.3, random_state=123)

In [8]:
def eval(model, X_test, y_test):
    y_pred = model.predict(X_test) # your predicted y values
    y_pred_int64 = y_pred.round(0).astype(int)
    # The root mean square logarithmic error
    msle = np.mean((np.log(y_pred_int64 + 1) - np.log(y_test + 1)) ** 2)
    rmsle = math.sqrt(msle)
    
    print ("RMSLE: %.4f" % rmsle)
    from sklearn.metrics import r2_score
    r2 = r2_score(y_pred, y_test)
    print ("R2 value: %.4f" % r2)
    return rmsle, r2

In [9]:
from sklearn import ensemble

model= ensemble.RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=500,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=1, warm_start=False)

model.fit(X_train, y_train)
eval(model, X_test, y_test)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  6.3min finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    0.2s


RMSLE: 0.5160
R2 value: -12.9170


[Parallel(n_jobs=12)]: Done 1000 out of 1000 | elapsed:    0.3s finished


(0.5160180810686386, -12.917024989425094)

In [ ]:
feature_scores_rdfr = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=True)
feature_scores_rdfr.plot(kind='barh', figsize=(20, 20))

In [ ]:
m = (df_target.columns != 'TARGET') & (df_target.columns != 'MemberID') & (df_target.columns != 'Year') & (df_target.columns != 'trainset')
X_submit = df_target.loc[:, m]
y_submit = df_target['TARGET']
y_submit.describe()

In [ ]:
X_submit_scl = scaler.transform(X_submit)

In [ ]:
yrdf_pred_submit = best_model.predict(X_submit_scl)

submission = pd.DataFrame()
submission['MemberID'] = df_target['MemberID']

yrdf_pred_submit = yrdf_pred_submit.clip(min=0)
yrdf_pred_submit = yrdf_pred_submit.clip(max=15)
#pred_y = pred_y.astype(int)

submission['DIH'] = yrdf_pred_submit
submission.groupby(['DIH']).size()
submission.to_csv('../4.Evaluation/randfr.csv',index=False)